Computing Introduction<br>
    Classical<br>
        Bit<br><br>
            0 or 1<br>
    Quantum<br>
        Qubit<br>
            0 or 1 or...both?<br>
            Superposition<br>
            Entanglement<br>
Gates Introduction<br>
    Classical<br>
        NAND<br>
        NOT<br>
        OR<br>
        AND<br>
        XOR<br>
        NOR<br>
        XNOR<br>
    Quantum<br>
        Hadamard<br>
$$\frac{1}{\sqrt{2}}\left[{\begin{array}{cc}1&1\\1&-1\\\end{array}}\right]$$
        Pauli-X<br>
$$\left[{\begin{array}{cc}0&1\\1&0\\\end{array}}\right]$$
        Pauli-Y<br>
$$\left[{\begin{array}{cc}0&-i\\i&0\\\end{array}}\right]$$
        Pauli-Z<br>
$$\left[{\begin{array}{cc}1&0\\0&-1\\\end{array}}\right]$$
        Phase<br>
$$\left[{\begin{array}{cc}1&0\\0&i\\\end{array}}\right]$$
        π/8<br>
$$\left[{\begin{array}{cc}1&0\\0&e^{i\pi/4}\\\end{array}}\right]$$
        Controlled-NOT (classical?)<br>
$$\left[{\begin{array}{cccc}1&0&0&0\\0&1&0&0\\0&0&0&1\\0&0&1&0\\\end{array}}\right]$$

To get started, we must import all necessary librarys and declare any variables used throughout the project

In [ ]:
# Initialization Stuff

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import plot_state_city
from PIL import Image

initial_state = [1, 0]

$$ \text{For this Bell state, } \alpha \text{ and } \beta \text{ are both } \frac{1}{\sqrt{2}} \text{ because } |\alpha|^2+|\beta|^2=1$$

$$ \text{This prepares a Bell state } \frac{1}{\sqrt{2}} |00⟩ + \frac{1}{\sqrt{2}} |11⟩$$

In [ ]:
# Bell State

# Matrix Math
q0 = np.array(initial_state)
q1 = np.array(initial_state)

def hadamard(q):
    h = np.array([[1, 1], 
                  [1, -1]])
    return np.matmul(q, h) * (2 ** -0.5)

def cnot(q0, q1):
    cx = np.array([[1, 0, 0, 0], 
                   [0, 1, 0, 0], 
                   [0, 0, 0, 1], 
                   [0, 0, 1, 0]])
    q = np.kron(q0, q1)
    return np.matmul(q, cx)

q0 = hadamard(q0)
q = cnot(q0, q1)

print(q)

# Qiskit
qc = QuantumCircuit(2)
qc.initialize(initial_state, 0)
qc.initialize(initial_state, 1)

qc.h(0)
qc.cx(0, 1)

qc.draw()
#simulate qiskit to show same result as matrix math !!!!!!!!!!!!!!!!!!!!!!

$$\text{Rx is }\left[{\begin{array}{cc}\cos(θ/2)&-1j*\sin(θ/2)\\-1j*\sin(θ/2)&\cos(θ/2)\\\end{array}}\right]$$

$$\text{This is a rotation around the X-axis of the Bloch sphere by } \theta \text{ radians}$$

In [ ]:
# Plot Two

counts = {'00': 0, '01': 0, '10': 0, '11': 0}

def rx(q, θ):
    rx = np.array([[np.cos(θ/2), -1j * np.sin(θ/2)], 
                  [-1j * np.sin(θ/2), np.cos(θ/2)]])
    return np.matmul(q, rx)

def cnot(q0, q1):
    cx = np.array([[1, 0, 0, 0], 
                   [0, 1, 0, 0], 
                   [0, 0, 0, 1], 
                   [0, 0, 1, 0]])
    q = np.kron(q0, q1)
    return np.matmul(q, cx)

for _ in range(1000):
    q0 = np.array(initial_state)
    q1 = np.array(initial_state)
    
    q0 = rx(q0, np.pi/4)
    q = cnot(q0, q1)
    
    counts[np.random.choice(['00', '01', '10', '11'], p=np.abs(q)**2)] += 1

print(counts)
plt.bar(['00', '01', '10', '11'], [counts['00'], counts['01'], counts['10'], counts['11']])

$$ \text{} $$

In [ ]:
# Shor Encoding

theta_list = np.linspace(0, 2*np.pi, 50)

def shor_encoding(theta, addErrors=False, multipleErrors=False):
    qc = QuantumCircuit(9,9)

    # Qubit 0 is being encoded
    qc.ry(theta, 0)

    qc.cx(0,3)
    qc.cx(0,6)

    qc.h(0)
    qc.h(3)
    qc.h(6)

    qc.cx(0,1)
    qc.cx(3,4)
    qc.cx(6,7)
    qc.cx(0,2)
    qc.cx(3,5)
    qc.cx(6,8)

    qc.barrier()

    # Random corruption of qubit 0
    corruptedQubit = 0

    # Random corruption of any single qubit
    corruptedQubit = np.random.randint(0, 9)
    
    if(addErrors):
        if(multipleErrors):
            for qubit in qc.qubits:
                np.random.choice([qc.x, qc.y, qc.z])(qubit)
        else:
            np.random.choice([qc.x, qc.y, qc.z])(corruptedQubit)
    
    qc.barrier()

    qc.cx(0,1)
    qc.cx(3,4)
    qc.cx(6,7)
    qc.cx(0,2)
    qc.cx(3,5)
    qc.cx(6,8)

    qc.ccx(1,2,0)
    qc.ccx(4,5,3)
    qc.ccx(7,8,6)

    qc.h(0)
    qc.h(3)
    qc.h(6)

    qc.cx(0,3)
    qc.cx(0,6)

    qc.ccx(3,6,0)

    qc.barrier()
    #qc.measure(range(9), range(9))
    qc.measure(0, 0)

    return qc

zero_counts = []
one_counts = []

# No Errors
for theta in theta_list:
    qc = shor_encoding(theta)
    qc.draw()
    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(transpile(qc, backend))
    counts = job.result().get_counts(qc)
    zero_count = 0
    one_count = 0
    
    for key in counts.keys():
        if(key[-1] == '0'):
            zero_count += counts.get(key)
        if(key[-1] == '1'):
            one_count += counts.get(key)
            
    zero_counts.append(zero_count)
    one_counts.append(one_count)

plt.plot(theta_list, zero_counts, label='0')
plt.plot(theta_list, one_counts, label='1')
plt.legend()
plt.show()

zero_counts = []
one_counts = []

# One Error
for theta in theta_list:
    qc = shor_encoding(theta, True)
    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(transpile(qc, backend))
    counts = job.result().get_counts(qc)
    zero_count = 0
    one_count = 0
    
    for key in counts.keys():
        if(key[-1] == '0'):
            zero_count += counts.get(key)
        if(key[-1] == '1'):
            one_count += counts.get(key)
            
    zero_counts.append(zero_count)
    one_counts.append(one_count)

plt.plot(theta_list, zero_counts, label='0')
plt.plot(theta_list, one_counts, label='1')
plt.legend()
plt.show()

zero_counts = []
one_counts = []

# Many Errors
for theta in theta_list:
    qc = shor_encoding(theta, True, True)
    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(transpile(qc, backend))
    counts = job.result().get_counts(qc)
    zero_count = 0
    one_count = 0
    
    for key in counts.keys():
        if(key[-1] == '0'):
            zero_count += counts.get(key)
        if(key[-1] == '1'):
            one_count += counts.get(key)
            
    zero_counts.append(zero_count)
    one_counts.append(one_count)

plt.plot(theta_list, zero_counts, label='0')
plt.plot(theta_list, one_counts, label='1')
plt.legend()
plt.show()

In [ ]:
# Bitstream

def shor_encoding(bit, addError=False, protectErrors=False):
    qc = QuantumCircuit(9,9)

    if(bit == 1):
        qc.x(0)

    qc.cx(0,3)
    qc.cx(0,6)

    qc.h(0)
    qc.h(3)
    qc.h(6)

    qc.cx(0,1)
    qc.cx(3,4)
    qc.cx(6,7)
    qc.cx(0,2)
    qc.cx(3,5)
    qc.cx(6,8)

    qc.barrier()

    # Random corruption of any single qubit
    corruptedQubit = np.random.randint(0, 9)
    
    if(addError):
        error = np.random.choice([qc.x, qc.y, qc.z])
        error(corruptedQubit)
    
    qc.barrier()

    qc.cx(0,1)
    qc.cx(3,4)
    qc.cx(6,7)
    qc.cx(0,2)
    qc.cx(3,5)
    qc.cx(6,8)

    qc.ccx(1,2,0)
    qc.ccx(4,5,3)
    qc.ccx(7,8,6)

    qc.h(0)
    qc.h(3)
    qc.h(6)

    qc.cx(0,3)
    qc.cx(0,6)

    qc.ccx(3,6,0)

    qc.barrier()
    
    if(not protectErrors):
        error(corruptedQubit)
    
    qc.measure(0, 0)

    job = backend.run(transpile(qc, backend))
    return job.result().get_counts(qc)

def string_to_binary(string):
    binary = ''.join(format(ord(char), 'b') for char in string)
    return binary

def binary_to_string(binary):
    chunks = [binary[i:i+7] for i in range(0, len(binary), 7)]
    decimals = [int(chunk, 2) for chunk in chunks]
    string = ''.join(chr(decimal) for decimal in decimals)
    return string

def bitstream(input_string, protectErrors=False):
    print('Input String: ' + input_string)
    
    input_bitstream = [int(bit) for bit in string_to_binary(input_string)]
    output_bitstream = ''
    for bit in input_bitstream:
        output = list(shor_encoding(bit, True, protectErrors).keys())[0]
        output_bitstream += output[len(output)-1]

    output_string = binary_to_string(output_bitstream)
    print('Output String: ' + output_string)
    
bitstream('Test')
bitstream('Test', True)

In [ ]:
# Eavesdropper

def create_bell_pair(qc, qubit, target):
    qc.h(qubit)
    qc.cx(qubit, target)

def superdense_encode(qc, bits, qubit):
    if bits == '00':
        pass
    elif bits == '01':
        qc.x(qubit)
    elif bits == '10':
        qc.z(qubit)
    elif bits == '11':
        qc.x(qubit)
        qc.z(qubit)
        
def superdense_decode(qc, qubit, target):
    qc.cx(qubit, target)
    qc.h(qubit)

# Didn't use this to demonstrate eavesdropping !!!!!!!!!!!!!
def show_image(image):
    for row in np.array([int(p) for p in image]).reshape(4, 4):
        print(' '.join(['X' if p == 1 else 'O' for p in row]))

def eavesdropper(eve):
    # 4 x 4
    image = '0000111100001111'
    
    for i in range(0, 16, 2):
        qc = QuantumCircuit(2,3)
        
        chunk = image[i:i+2]
        
        create_bell_pair(qc, 0, 1)
        superdense_encode(qc, chunk, 0)
        
        if(eve):
            qc.measure(1, 2)
    
        superdense_decode(qc, 0, 1)
        qc.measure(1, 1)
        
        job = backend.run(transpile(qc, backend))
        counts = job.result().get_counts(qc)
    
        print('Alice\'s Sent Message: ' + chunk)
        print('Bob\'s Received Counts: ' + str(counts))


eavesdropper(False)
print()
eavesdropper(True)

In [ ]:
# BB84

def gen_key(num, a_states, a_basis, b_basis):
    qc = QuantumCircuit(num)
    
    for i in range(num):
        if a_states[i] == 1:
            qc.x(i)
        if a_basis[i] == 1:
            qc.h(i)
    
    qc.barrier()
    
    for j in range(num):
        if b_basis[j] == 1:
            qc.h(j)
            
    qc.measure_all()
    
    job = backend.run(transpile(qc, backend), shots=2)
    counts = job.result().get_counts(qc)
    randomChoice = np.random.choice(list(counts.keys()))
    
    key = ''
    for i in range(num):
        if a_basis[i] == b_basis[i]:
            key += str(randomChoice[i])
            
    return key

def bb84(message, eve=False):
    num = 24 * len(message)
    
    a_states = np.random.randint(2, size=num)
    a_basis = np.random.randint(2, size=num)
    b_basis = np.random.randint(2, size=num)
    
    #print("Alice States: " + str(a_states))
    #print("Alice Basis: " + str(a_basis))
    #print("Bob Basis: " + str(b_basis))
    
    key = gen_key(num, a_states, a_basis, b_basis)
    
    if eve:
        e_basis = np.random.randint(2, size=num)
        e_states = gen_key(num, a_states, a_basis, e_basis)
        
        for i in range(num):
            if(e_basis[i] == b_basis[i]):
                b_basis[i] = 1 - b_basis[i]
                a_states[i] = e_states[i]
    
    
    
    encrypted = ''
    for i in range(len(message)):
        keyByte = int(key[8*i:8*i+8],2)
        #print(keyByte)
        encrypted += chr(ord(message[i]) ^ keyByte)
        
    print("Encrypted: " + encrypted)
    
    decrypted = ''
    for i in range(len(encrypted)):
        keyByte = int(key[8*i:8*i+8],2)
        decrypted += chr(ord(encrypted[i]) ^ keyByte)
        
    print("Decrypted: " + decrypted)
    
message = "abcd"

bb84(message, True)

Future Projects

Credit


Neilsen and Chuang
https://www.forbes.com/sites/technology/article/what-is-a-qubit/